In [5]:
from allensdk.api.queries.rma_api import RmaApi

rma = RmaApi()

data = rma.model_query('Atlas',
                        criteria="[name$il'*Mouse*']")


In [6]:
associations = ''.join(['[id$eq1]',
                        'structure_graph(ontology),',
                        'graphic_group_labels'])

atlas_data = rma.model_query('Atlas',
                                include=associations,
                                criteria=associations,
                                only=['atlases.id',
                                    'atlases.name',
                                    'atlases.image_type',
                                    'ontologies.id',
                                    'ontologies.name',
                                    'structure_graphs.id',
                                    'structure_graphs.name',
                                    'graphic_group_labels.id',
                                    'graphic_group_labels.name'])

In [7]:
# http://api.brain-map.org/api/v2/data.json
schema = rma.get_schema()
for entry in schema:
    data_description = entry['DataDescription']
    clz = list(data_description.keys())[0]
    info = list(data_description.values())[0]
    fields = info['fields']
    associations = info['associations']
    table = info['table']
    print("class: %s" % (clz))
    print("fields: %s" % (','.join(f['name'] for f in fields)))
    print("associations: %s" % (','.join(a['name'] for a in associations)))
    print("table: %s\n" % (table))

class: ManualAnnotation
fields: id,structure_id,section_data_set_id,annotation_plan_name,intensity_call,density_call,pattern_call,mixed_population_call,comment
associations: structure,section_data_set
table: manual_annotations

class: NeuronalModel
fields: id,name,neuronal_model_template_id,specimen_id,neuron_reconstruction_id
associations: neuronal_model_template,neuron_reconstruction,neuronal_model_runs,specimen,well_known_files
table: neuronal_models

class: Gene
fields: id,name,entrez_id,acronym,organism_id,chromosome_id,homologene_id,type,legacy_ensembl_gene_id,ensembl_id,alias_tags,sphinx_id,original_symbol,original_name,version_status,genomic_reference_update_id,reference_genome_id
associations: organism,transcripts,gene_classifications,chromosome,genomic_reference_update,reference_genome,gene_aliases,probes,gene_lookups,gene_classification_counts,products,data_sets,gene_associations,transgenic_lines,molbio_authority,data_points,homologous_gene_associations,homologous_genes,ense

In [8]:
import pandas as pd

structures = pd.DataFrame(
    rma.model_query('Structure',
                    criteria='[graph_id$eq1]',
                    num_rows='all'))

In [9]:
names_and_acronyms = structures.loc[:,['name', 'acronym']]

In [10]:
mea = structures[structures.acronym == 'MEA']
mea_id = mea.iloc[0,:].id
mea_children = structures[structures.parent_structure_id == mea_id]
print(mea_children['name'])

612     Medial amygdalar nucleus, posteroventral part
921      Medial amygdalar nucleus, posterodorsal part
1244     Medial amygdalar nucleus, anteroventral part
1275      Medial amygdalar nucleus, anterodorsal part
Name: name, dtype: object


In [11]:
criteria_string = "structure_sets[name$eq'Mouse Connectivity - Summary']"
include_string = "ontology"
summary_structures = \
    pd.DataFrame(
        rma.model_query('Structure',
                        criteria=criteria_string,
                        include=include_string,
                        num_rows='all'))
ontologies = \
    pd.DataFrame(
        list(summary_structures.ontology)).drop_duplicates()
flat_structures_dataframe = summary_structures.drop(['ontology'], axis=1)

In [12]:
summary_structures[['id',
                    'parent_structure_id',
                    'acronym']].to_csv('summary_structures.csv',
                                        index_label='structure_id')
reread = pd.read_csv('summary_structures.csv')

In [13]:
for id, name, parent_structure_id in summary_structures[['name',
                                                            'parent_structure_id']].itertuples():
    print("%d %s %d" % (id, name, parent_structure_id))

0 Periaqueductal gray 323
1 Arcuate hypothalamic nucleus 157
2 Orbital area, medial part 714
3 Lateral septal nucleus, ventral part 242
4 Posterodorsal preoptic nucleus 141
5 Area postrema 386
6 Flocculus 1073
7 Subparafascicular area 864
8 Oculomotor nucleus 323
9 Posterolateral visual area 669
10 fiber tracts 997
11 Entorhinal area, medial part, dorsal zone 909
12 Paraventricular hypothalamic nucleus 157
13 Intercalated amygdalar nucleus 278
14 Intermediodorsal nucleus of the thalamus 444
15 Superior colliculus, sensory related 339
16 Supraoptic nucleus 157
17 Central linear nucleus raphe 165
18 Reticular nucleus of the thalamus 856
19 Dorsal nucleus raphe 165
20 Ventral cochlear nucleus 607
21 Posterior pretectal nucleus 1100
22 Retrosplenial area, dorsal part 254
23 Laterointermediate area 669
24 Bed nucleus of the anterior commissure 809
25 Nucleus sagulum 339
26 Gigantocellular reticular nucleus 370
27 Orbital area, lateral part 714
28 Perirhinal area 315
29 Presubiculum 822
30 P